# Interactive dynamic simulations: create your own mutants to study mRNA stability

Author: Anna B. Matuszyńska <a href="https://orcid.org/0000-0003-0882-6088"><img src="https://orcid.org/assets/vectors/orcid.logo.icon.svg" width=15></a>

In this notebook you can **create your own strain** and investigate the effect of:
- initial mRNA concentration, 
- rate of mRNA synthesis and 
- mRNA degradation 
on the overall mRNA dynamics.

### How to use the notebook? 
To do the first run put a cursor on the cell below, the one with a lot of code in it and **execte it**: either by hitting the button Run from the Jupyter Notebook panel above this text, or by clicking Shift+Enter. Execution of this cell will lead to printing the message to you and the coursor will automatically move to the next cell.

Execute it too! What you will see is the three sliders poping up on your screen: with these sliders you are able to  select the desigered values for the three parameters affecting mRNA dynamics without really needing to code anything! Once you select the values of interest using sliders hit the *Run Simulation* button below.

You can save every result of your simulation by right clicking the figure and saving it as.

**Have fun!**

In [1]:
from model import m
import numpy as np
from modelbase.ode import Model, Simulator, mca

def decay(halftime):
    """ return the rate of mRNA decay for the given half-life"""
    return np.log(2)/halftime

%matplotlib inline

import numpy as np
import pandas

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from modelbase.ode import Model, Simulator, mca
from modelbase.ode import ratelaws as rl

m = Model()

m.add_compounds(["mRNA"])
m.add_parameters({'kSynth': 1, 'kDecay': decay(0.0578)})

def constant(k):
    return k

def mass_action_1s(s, kp):
    """Irreversible mass-action function with 1 substrate
    Arguments:
    s -> Substrate
    kp -> positive rate constant 
    """
    return kp * s

m.add_reaction_from_args(
    rate_name = "vSynth", #This should be a unique name that helps you identify the appropriate reaction
    function = constant, #This is the appropriate function you created beforehand
    stoichiometry = {"mRNA": 1}, #This is a dictionary of the compounds with their respective stochiometry in this specific reaction
    args = ["kSynth"] #this is a list of all arguments passed to the function, in their respective oder
)

m.add_reaction_from_args(
    rate_name = "vDecay",
    function = mass_action_1s,
    stoichiometry = {"mRNA": -1},
    args = ["mRNA", "kDecay"],
    reversible = False
)

print('Well done, the model has been created and now you can run unlimited simulations with it')

Well done, the model has been created and now you can run unlimited simulations with it


In [10]:
import ipywidgets as widgets
from IPython.display import display

widgets.interact_manual.opts['manual_name'] = 'Run simulation'

@widgets.interact_manual(
    y0= widgets.IntSlider(
    value=36,
    min=0,
    max=1000,
    step=1,
    description='[mRNA] at t0',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
), synthesis=widgets.IntSlider(
    value=36,
    min=0,
    max=1000,
    step=1,
    description='synthesis',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
), halftime=(1., 180.))
def plot(y0=36., synthesis=30, halftime=8, grid=True):
    s = Simulator(m)
    y0 = {'mRNA': y0}
    m.update_parameters({'kSynth': synthesis, 'kDecay': decay(halftime)})
    s.initialise(y0)
    t, y = s.simulate(4*60) #Actually simulating the model until the given time point. The specifc time steps are stored as an array in the variable t
                                #and the different concentrations in a nested array
    findhalf = min([i for i in range(len(y)) if y[i] >= 0.5 * y[-1]])
    
    plt.plot(t[findhalf], 
             y[findhalf], marker='o', color='r', markersize=5)
    
    plt.axhline(y=y[findhalf], xmin=0, xmax=1, color='r', linestyle = "--")
    plt.axvline(x=t[findhalf], color='r', linestyle = "--")
    
    plt.plot(t,y, color='r')
    plt.xlabel("Time/min.")
    plt.ylabel("Concentration [au]") #modelbase gives us the option to plot immediately through our Simulator() object,
                                                                        #but we could have also done it with the t and y variables appointed earlier.
    print('Results of the simulation for the selected parameters: rate of synthesis:', str(synthesis), 
          ', initial concentration of mRNA set to: ', str(y0), 'and halftime of: ', str(halftime))

interactive(children=(IntSlider(value=36, continuous_update=False, description='[mRNA] at t0', max=1000), IntS…